In [ ]:
from pathlib import Path
import sys
import os
from dotenv import find_dotenv
PROJECT_ROOT = Path(find_dotenv()).parent
sys.path.append(str(PROJECT_ROOT.joinpath('src')))
from neuprint import Client, fetch_neurons, fetch_adjacencies, NeuronCriteria as NC, fetch_custom, fetch_roi_hierarchy
c = Client('neuprint-cns.janelia.org', dataset='cns', token=os.environ['NEUPRINT_TOKEN'])
import pandas as pd

In [ ]:
neuron_type_name = "DNc02"
instance_name = "DNc02(SIFa)_R"

In [ ]:
criteria = NC(instance=instance_name, regex=False)
neurons_df, roi_df = fetch_neurons(criteria)

In [ ]:
# Add neurotransmitter fields via separate query if neurons were found
if not neurons_df.empty:
    body_ids = neurons_df['bodyId'].tolist()
    body_ids_str = "[" + ", ".join(str(bid) for bid in body_ids) + "]"

    # Query for neurotransmitter and class fields
    nt_query = f"""
    UNWIND {body_ids_str} as target_body_id
    MATCH (n:Neuron {{bodyId: target_body_id}})
    RETURN
        target_body_id as bodyId,
        n.consensusNt as consensusNt,
        n.celltypePredictedNt as celltypePredictedNt,
        n.celltypePredictedNtConfidence as celltypePredictedNtConfidence,
        n.celltypeTotalNtPredictions as celltypeTotalNtPredictions,
        n.class as cellClass,
        n.subclass as cellSubclass,
        n.superclass as cellSuperclass
    """


    nt_df = c.fetch_custom(nt_query)
    if not nt_df.empty:
        # Merge neurotransmitter data with neurons_df
        neurons_df = neurons_df.merge(nt_df, on='bodyId', how='left')


In [ ]:
neurons_df

In [ ]:
neurons_df.roiInfo.to_dict()

 'LO_L_layer_1': {'post': 41,
   'pre': 139,
   'downstream': 404,
   'upstream': 41,
   'synweight': 445},
  'LO_L_layer_2': {'post': 26,
   'pre': 60,
   'downstream': 165,
   'upstream': 26,
   'synweight': 191},
  'LO_L_layer_7': {'pre': 2, 'downstream': 4, 'synweight': 4},

'ME_L_layer_02': {'pre': 1, 'downstream': 2, 'synweight': 2},
  'ME_L_layer_03': {'post': 6,
   'pre': 25,
   'downstream': 84,
   'upstream': 6,
   'synweight': 90},
  'ME_L_layer_04': {'post': 17,
   'pre': 37,
   'downstream': 120,
   'upstream': 17,
   'synweight': 137},
  'ME_L_layer_05': {'post': 28,
   'pre': 23,
   'downstream': 72,
   'upstream': 28,
   'synweight': 100},
  'ME_L_layer_06': {'post': 25,
   'pre': 19,
   'downstream': 62,
   'upstream': 25,
   'synweight': 87},
  'ME_L_layer_08': {'post': 2,
   'pre': 9,
   'downstream': 29,
   'upstream': 2,
   'synweight': 31},
  'ME_L_layer_09': {'post': 41,
   'pre': 91,
   'downstream': 272,
   'upstream': 41,
   'synweight': 313},
  'ME_L_layer_10': {'post': 62,
   'pre': 110,
   'downstream': 311,
   'upstream': 62,
   'synweight': 373},

In [ ]:
roi_df

In [ ]:
mask = roi_df['roi'].str.contains('layer', na=False)
layer_rows = roi_df[mask]

In [ ]:
layer_rows

In [ ]:
sum(layer_rows['post'])

Add query to check post synapses between this instance type & neurons versus segments

In [ ]:
cql = f"""
MATCH (n:Neuron)<-[e:ConnectsTo]-(m:Neuron)
WHERE n.instance = '{instance_name}'
AND (NOT m.type IS NULL AND NOT m.instance IS NULL)
RETURN n.bodyId, m.bodyId, m.status, apoc.convert.fromJsonMap(e.roiInfo) as roiInfo
"""

In [ ]:
df = c.fetch_custom(cql)
df

In [ ]:
df['keys(e)'].roiIno